## Cross-validation and Ensembling 5 UNETR folds trained and saved as 5 separate model files

In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
# !python -c "import monai" || pip install -q "monai-weekly[ignite, nibabel, tqdm]"
!pip install -q "monai-weekly[nibabel, tqdm, einops]"
!python -c "import matplotlib" || pip install -q matplotlib
%matplotlib inline

In [3]:
from abc import ABC, abstractmethod
import logging
import os
import tempfile
import shutil
import sys

import nibabel as nib
import numpy as np
import torch

from monai.apps import CrossValidation
from monai.config import print_config
from monai.data import CacheDataset, DataLoader, create_test_image_3d
from monai.engines import EnsembleEvaluator, SupervisedEvaluator, SupervisedTrainer
from monai.handlers import MeanDice, StatsHandler, ValidationHandler, from_engine
from monai.inferers import SimpleInferer, SlidingWindowInferer
from monai.losses import DiceLoss
from monai.networks.nets import UNet, SwinUNETR, UNETR
from monai.metrics import DiceMetric

from monai.transforms import (
    Activationsd,
    EnsureChannelFirstd,
    AsDiscreted,
    Compose,
    LoadImaged,
    MeanEnsembled,
    RandCropByPosNegLabeld,
    RandRotate90d,
    ScaleIntensityd,
    EnsureTyped,
    VoteEnsembled,
)
from monai.utils import set_determinism

print_config()

MONAI version: 0.10.dev2231
Numpy version: 1.23.1
Pytorch version: 1.12.0+cu102
MONAI flags: HAS_EXT = False, USE_COMPILED = False, USE_META_DICT = False
MONAI rev id: 8236821c099e5997f7ad0410c5374e4472f4ed14
MONAI __file__: /home/adminbthomas/miniconda3/envs/PT02/lib/python3.9/site-packages/monai/__init__.py

Optional dependencies:
Pytorch Ignite version: 0.4.9
Nibabel version: 4.0.1
scikit-image version: 0.19.3
Pillow version: 9.2.0
Tensorboard version: 2.9.1
gdown version: 4.5.1
TorchVision version: 0.13.0+cu102
tqdm version: 4.64.0
lmdb version: 1.3.0
psutil version: 5.9.1
pandas version: 1.4.3
einops version: 0.4.1
transformers version: 4.20.1
mlflow version: 1.27.0
pynrrd version: NOT INSTALLED or UNKNOWN VERSION.

For details about installing the optional dependencies, please visit:
    https://docs.monai.io/en/latest/installation.html#installing-the-recommended-dependencies



In [4]:
root_dir = "/home/adminbthomas/Downloads/ensemble"
print(root_dir)

/home/adminbthomas/Downloads/ensemble


In [5]:
set_determinism(seed=0)
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
device = torch.device("cuda:0")

In [6]:
model_dir = os.path.join(root_dir,"unetr/")

In [7]:
models=[]

In [8]:
for i in range(5):
    model = UNETR(
        in_channels=1,
        out_channels=6,
        img_size=(96, 96, 16),
        feature_size=16,
        hidden_size=768,
        mlp_dim=3072,
        num_heads=12,
        pos_embed="perceptron",
        norm_name="instance",
        res_block=True,
        dropout_rate=0.0,
        ).to(device)
    model.load_state_dict(torch.load(os.path.join(model_dir, f"TestMR09_unetrensemble_fold{i}.pth")))
    print(f'Appending Fold{i}')
    model.eval()
    models.append(model)

Appending Fold0
Appending Fold1
Appending Fold2
Appending Fold3
Appending Fold4


In [9]:
import glob
from monai.data import Dataset
from monai.handlers.utils import from_engine
from monai.transforms import (
    AsDiscrete,
    AsDiscreted,
    EnsureChannelFirstd,
    Compose,
    CropForegroundd,
    LoadImaged,
    Orientationd,
    RandCropByPosNegLabeld,
    SaveImaged,
    ScaleIntensityRanged,
    Spacingd,
    SplitChanneld,
    EnsureTyped,
    EnsureType,
    Invertd,
)

test_images = sorted(
    glob.glob(os.path.join(root_dir, "data/imagesTs", "*T2W.nii.gz")))
# test_labels = sorted(
#     glob.glob(os.path.join(data_dir, "data/imagesTs/outs_binarised/", "seg*.nii.gz")))

# test_data = [{"image": image,"label": label} for image,label in zip(test_images, test_labels)]
test_data = [{"image": image} for image in test_images]

test_org_transforms = Compose(
    [
        LoadImaged(keys="image"),
        EnsureChannelFirstd(keys="image"),
        Orientationd(keys=["image"], axcodes="RAS"),
        Spacingd(keys=["image"], pixdim=(
            1.5, 1.5, 4.0), mode="bilinear"),
        ScaleIntensityRanged(
            keys=["image"], a_min=0, a_max=250,
            b_min=0.0, b_max=1.0, clip=False,
        ),
        CropForegroundd(keys=["image"], source_key="image"),
        EnsureTyped(keys="image"),
    ]
)

test_org_ds = Dataset(
    data=test_data, transform=test_org_transforms)

test_loader = DataLoader(test_org_ds, batch_size=1, num_workers=4)

In [10]:
test_data

[{'image': '/home/adminbthomas/Downloads/ensemble/data/imagesTs/A002_v1_T2W.nii.gz'},
 {'image': '/home/adminbthomas/Downloads/ensemble/data/imagesTs/A002_v2_T2W.nii.gz'},
 {'image': '/home/adminbthomas/Downloads/ensemble/data/imagesTs/A002_v3_T2W.nii.gz'},
 {'image': '/home/adminbthomas/Downloads/ensemble/data/imagesTs/A003_v1_T2W.nii.gz'},
 {'image': '/home/adminbthomas/Downloads/ensemble/data/imagesTs/A003_v2_T2W.nii.gz'},
 {'image': '/home/adminbthomas/Downloads/ensemble/data/imagesTs/A009_v1_T2W.nii.gz'},
 {'image': '/home/adminbthomas/Downloads/ensemble/data/imagesTs/A009_v2_T2W.nii.gz'},
 {'image': '/home/adminbthomas/Downloads/ensemble/data/imagesTs/A011_v1_T2W.nii.gz'},
 {'image': '/home/adminbthomas/Downloads/ensemble/data/imagesTs/A011_v2_T2W.nii.gz'},
 {'image': '/home/adminbthomas/Downloads/ensemble/data/imagesTs/A013_v1_T2W.nii.gz'},
 {'image': '/home/adminbthomas/Downloads/ensemble/data/imagesTs/A013_v2_T2W.nii.gz'},
 {'image': '/home/adminbthomas/Downloads/ensemble/data

In [16]:
def ensemble_inference(post_transforms, models):
    evaluator = EnsembleEvaluator(
        device=device,
        val_data_loader=test_loader,
        pred_keys=["pred0", "pred1", "pred2", "pred3", "pred4"],
        networks=models,
        inferer=SlidingWindowInferer(roi_size=(96, 96, 16), sw_batch_size=4, overlap=0.8),
        postprocessing=post_transforms,
    )
    evaluator.run()

In [17]:
mean_post_transforms = Compose(
    [
        EnsureTyped(keys=["pred0", "pred1", "pred2", "pred3", "pred4"]),
        MeanEnsembled(
            keys=["pred0", "pred1", "pred2", "pred3", "pred4"],
            output_key="pred",
            # in this particular example, we use validation metrics as weights
            weights=[0.9108, 0.8404, 0.9049, 0.8504, 0.8981],
        ),
        Invertd(
        keys="pred",
        transform=test_org_transforms,
        orig_keys="image",
        meta_keys="pred_meta_dict",
        orig_meta_keys="image_meta_dict",
        meta_key_postfix="meta_dict",
        nearest_interp=False,
        to_tensor=True,
        ),
        AsDiscreted(keys="pred", argmax=True, to_onehot=6),
        SaveImaged(keys="pred", meta_keys="pred_meta_dict", output_dir="./outMR11wtAvgAll", output_postfix="seg", resample=False, separate_folder=False),
    ]
)
ensemble_inference(mean_post_transforms, models)

INFO:ignite.engine.engine.EnsembleEvaluator:Engine run resuming from iteration 0, epoch 0 until 1 epochs


Modifying image pixdim from [1.4648438 1.4648438 4.        0.       ] to [  1.46484375   1.46484375   4.         268.45717811]
Modifying image pixdim from [1.4648438 1.4648438 4.        0.       ] to [  1.46484375   1.46484375   4.         268.45717811]
Modifying image pixdim from [1.4648438 1.4648438 4.        0.       ] to [  1.46484375   1.46484375   4.         268.45717811]
Modifying image pixdim from [1.4648438 1.4648438 4.        0.       ] to [  1.46484375   1.46484375   4.         268.45717811]


2024-03-30 19:28:18,747 INFO image_writer.py:194 - writing: outMR11wtAvgAll/A002_v1_T2W_seg.nii.gz
2024-03-30 19:28:27,787 INFO image_writer.py:194 - writing: outMR11wtAvgAll/A002_v2_T2W_seg.nii.gz
2024-03-30 19:28:36,878 INFO image_writer.py:194 - writing: outMR11wtAvgAll/A002_v3_T2W_seg.nii.gz
2024-03-30 19:28:45,999 INFO image_writer.py:194 - writing: outMR11wtAvgAll/A003_v1_T2W_seg.nii.gz
2024-03-30 19:28:55,311 INFO image_writer.py:194 - writing: outMR11wtAvgAll/A003_v2_T2W_seg.nii.gz
2024-03-30 19:29:04,572 INFO image_writer.py:194 - writing: outMR11wtAvgAll/A009_v1_T2W_seg.nii.gz
2024-03-30 19:29:13,851 INFO image_writer.py:194 - writing: outMR11wtAvgAll/A009_v2_T2W_seg.nii.gz
2024-03-30 19:29:22,968 INFO image_writer.py:194 - writing: outMR11wtAvgAll/A011_v1_T2W_seg.nii.gz
2024-03-30 19:29:32,132 INFO image_writer.py:194 - writing: outMR11wtAvgAll/A011_v2_T2W_seg.nii.gz
2024-03-30 19:29:41,425 INFO image_writer.py:194 - writing: outMR11wtAvgAll/A013_v1_T2W_seg.nii.gz
2024-03-30

Modifying image pixdim from [1.4648438 1.4648438 8.        0.       ] to [  1.46484375   1.46484375   8.         281.03604124]


2024-03-30 19:30:35,759 INFO image_writer.py:194 - writing: outMR11wtAvgAll/A016_v1_T2W_seg.nii.gz


...

Modifying image pixdim from [1.4648438 1.4648438 8.        0.       ] to [  1.46484375   1.46484375   8.         281.03604124]


2024-03-30 19:30:54,207 INFO image_writer.py:194 - writing: outMR11wtAvgAll/A017_v1_T2W_seg.nii.gz


Modifying image pixdim from [1.4648438 1.4648438 8.        0.       ] to [  1.46484375   1.46484375   8.         281.03604124]


2024-03-30 19:31:03,431 INFO image_writer.py:194 - writing: outMR11wtAvgAll/A017_v2_T2W_seg.nii.gz
2024-03-30 19:31:12,672 INFO image_writer.py:194 - writing: outMR11wtAvgAll/A018_v1_T2W_seg.nii.gz
2024-03-30 19:31:21,896 INFO image_writer.py:194 - writing: outMR11wtAvgAll/A018_v2_T2W_seg.nii.gz
2024-03-30 19:31:31,176 INFO image_writer.py:194 - writing: outMR11wtAvgAll/A021_v1_T2W_seg.nii.gz
2024-03-30 19:31:40,450 INFO image_writer.py:194 - writing: outMR11wtAvgAll/A021_v2_T2W_seg.nii.gz
2024-03-30 19:32:07,408 INFO image_writer.py:194 - writing: outMR11wtAvgAll/A022_v1_T2W_seg.nii.gz
2024-03-30 19:32:34,400 INFO image_writer.py:194 - writing: outMR11wtAvgAll/A022_v2_T2W_seg.nii.gz
2024-03-30 19:33:01,420 INFO image_writer.py:194 - writing: outMR11wtAvgAll/A023_v1_T2W_seg.nii.gz
2024-03-30 19:33:28,450 INFO image_writer.py:194 - writing: outMR11wtAvgAll/A023_v2_T2W_seg.nii.gz
2024-03-30 19:33:37,738 INFO image_writer.py:194 - writing: outMR11wtAvgAll/A024_v1_T2W_seg.nii.gz
2024-03-30

2024-03-30 19:52:29,783 INFO image_writer.py:194 - writing: outMR11wtAvgAll/A074_v2_T2W_seg.nii.gz
2024-03-30 19:52:37,608 INFO image_writer.py:194 - writing: outMR11wtAvgAll/A076_v1_T2W_seg.nii.gz
2024-03-30 19:52:46,892 INFO image_writer.py:194 - writing: outMR11wtAvgAll/A076_v2_T2W_seg.nii.gz
INFO:ignite.engine.engine.EnsembleEvaluator:Epoch[1] Complete. Time taken: 00:24:38
INFO:ignite.engine.engine.EnsembleEvaluator:Engine run complete. Time taken: 00:24:38


In [ ]:
vote_post_transforms = Compose(
    [
        EnsureTyped(keys=["pred0", "pred1", "pred2", "pred3", "pred4"]),
        Activationsd(keys=["pred0", "pred1", "pred2", "pred3", "pred4"], sigmoid=True),
        # transform data into discrete before voting
        AsDiscreted(keys=["pred0", "pred1", "pred2", "pred3", "pred4"], argmax=True, to_onehot=6),
        VoteEnsembled(keys=["pred0", "pred1", "pred2", "pred3", "pred4"], 
                      output_key="pred"),
        Invertd(
        keys="pred",
        transform=test_org_transforms,
        orig_keys="image",
        meta_keys="pred_meta_dict",
        orig_meta_keys="image_meta_dict",
        meta_key_postfix="meta_dict",
        nearest_interp=False,
        to_tensor=True,
        ),
        AsDiscreted(keys="pred", argmax=True, to_onehot=6),
        SaveImaged(keys="pred", meta_keys="pred_meta_dict", output_dir="./outMR11voteAll", output_postfix="seg", resample=False, separate_folder=False),
    ]
)
ensemble_inference(vote_post_transforms, models)

INFO:ignite.engine.engine.EnsembleEvaluator:Engine run resuming from iteration 0, epoch 0 until 1 epochs


Modifying image pixdim from [1.4648438 1.4648438 4.        0.       ] to [  1.46484375   1.46484375   4.         268.45717811]
Modifying image pixdim from [1.4648438 1.4648438 4.        0.       ] to [  1.46484375   1.46484375   4.         268.45717811]
Modifying image pixdim from [1.4648438 1.4648438 4.        0.       ] to [  1.46484375   1.46484375   4.         268.45717811]
Modifying image pixdim from [1.4648438 1.4648438 4.        0.       ] to [  1.46484375   1.46484375   4.         268.45717811]


2024-03-30 15:46:16,591 INFO image_writer.py:194 - writing: outMR11voteAll/A002_v1_T2W_seg.nii.gz
2024-03-30 15:46:25,628 INFO image_writer.py:194 - writing: outMR11voteAll/A002_v2_T2W_seg.nii.gz
2024-03-30 15:46:34,632 INFO image_writer.py:194 - writing: outMR11voteAll/A002_v3_T2W_seg.nii.gz
2024-03-30 15:46:43,715 INFO image_writer.py:194 - writing: outMR11voteAll/A003_v1_T2W_seg.nii.gz
2024-03-30 15:46:52,899 INFO image_writer.py:194 - writing: outMR11voteAll/A003_v2_T2W_seg.nii.gz
2024-03-30 15:47:02,104 INFO image_writer.py:194 - writing: outMR11voteAll/A009_v1_T2W_seg.nii.gz
2024-03-30 15:47:11,352 INFO image_writer.py:194 - writing: outMR11voteAll/A009_v2_T2W_seg.nii.gz
2024-03-30 15:47:20,671 INFO image_writer.py:194 - writing: outMR11voteAll/A011_v1_T2W_seg.nii.gz
2024-03-30 15:47:29,899 INFO image_writer.py:194 - writing: outMR11voteAll/A011_v2_T2W_seg.nii.gz
2024-03-30 15:47:39,039 INFO image_writer.py:194 - writing: outMR11voteAll/A013_v1_T2W_seg.nii.gz
2024-03-30 15:47:46,

Modifying image pixdim from [1.4648438 1.4648438 8.        0.       ] to [  1.46484375   1.46484375   8.         281.03604124]


2024-03-30 15:48:33,565 INFO image_writer.py:194 - writing: outMR11voteAll/A016_v1_T2W_seg.nii.gz


...

2024-03-30 15:48:42,770 INFO image_writer.py:194 - writing: outMR11voteAll/A016_v2_T2W_seg.nii.gz


Modifying image pixdim from [1.4648438 1.4648438 8.        0.       ] to [  1.46484375   1.46484375   8.         281.03604124]


2024-03-30 15:48:51,966 INFO image_writer.py:194 - writing: outMR11voteAll/A017_v1_T2W_seg.nii.gz


Modifying image pixdim from [1.4648438 1.4648438 8.        0.       ] to [  1.46484375   1.46484375   8.         281.03604124]


2024-03-30 15:49:01,167 INFO image_writer.py:194 - writing: outMR11voteAll/A017_v2_T2W_seg.nii.gz
2024-03-30 15:49:10,380 INFO image_writer.py:194 - writing: outMR11voteAll/A018_v1_T2W_seg.nii.gz
2024-03-30 15:49:19,625 INFO image_writer.py:194 - writing: outMR11voteAll/A018_v2_T2W_seg.nii.gz
2024-03-30 15:49:28,869 INFO image_writer.py:194 - writing: outMR11voteAll/A021_v1_T2W_seg.nii.gz
2024-03-30 15:49:38,104 INFO image_writer.py:194 - writing: outMR11voteAll/A021_v2_T2W_seg.nii.gz
2024-03-30 15:50:05,017 INFO image_writer.py:194 - writing: outMR11voteAll/A022_v1_T2W_seg.nii.gz
2024-03-30 15:50:31,945 INFO image_writer.py:194 - writing: outMR11voteAll/A022_v2_T2W_seg.nii.gz
2024-03-30 15:50:58,940 INFO image_writer.py:194 - writing: outMR11voteAll/A023_v1_T2W_seg.nii.gz
2024-03-30 15:51:25,930 INFO image_writer.py:194 - writing: outMR11voteAll/A023_v2_T2W_seg.nii.gz
2024-03-30 15:51:35,204 INFO image_writer.py:194 - writing: outMR11voteAll/A024_v1_T2W_seg.nii.gz
2024-03-30 15:51:44,

In [18]:
!tar -cvzf outMR11wtAvgAll.tar.gz outMR11wtAvgAll

outMR11wtAvgAll/
outMR11wtAvgAll/A026_v1_T2W_seg.nii.gz
outMR11wtAvgAll/A011_v2_T2W_seg.nii.gz
outMR11wtAvgAll/A003_v1_T2W_seg.nii.gz
outMR11wtAvgAll/A034_v1_T2W_seg.nii.gz
outMR11wtAvgAll/A017_v1_T2W_seg.nii.gz
outMR11wtAvgAll/A057_v2_T2W_seg.nii.gz
outMR11wtAvgAll/A031_v1_T2W_seg.nii.gz
outMR11wtAvgAll/A033_v3_T2W_seg.nii.gz
outMR11wtAvgAll/A011_v1_T2W_seg.nii.gz
outMR11wtAvgAll/A071_v2_T2W_seg.nii.gz
outMR11wtAvgAll/A048_v2_T2W_seg.nii.gz
outMR11wtAvgAll/A028_v1_T2W_seg.nii.gz
outMR11wtAvgAll/A068_v1_T2W_seg.nii.gz
outMR11wtAvgAll/A032_v2_T2W_seg.nii.gz
outMR11wtAvgAll/A071_v1_T2W_seg.nii.gz
outMR11wtAvgAll/A049_v1_T2W_seg.nii.gz
outMR11wtAvgAll/A061_v2_T2W_seg.nii.gz
outMR11wtAvgAll/A026_v2_T2W_seg.nii.gz
outMR11wtAvgAll/A043_v2_T2W_seg.nii.gz
outMR11wtAvgAll/A050_v1_T2W_seg.nii.gz
outMR11wtAvgAll/A063_v1_T2W_seg.nii.gz
outMR11wtAvgAll/A046_v2_T2W_seg.nii.gz
outMR11wtAvgAll/A041_v2_T2W_seg.nii.gz
outMR11wtAvgAll/A003_v2_T2W_seg.nii.gz
outMR11wtAvgAll/A033_v1_T2W_seg.nii.gz
outMR11w

In [ ]:
# !tar -cvzf outMR11meanAll.tar.gz outMR11meanAll